## EDA for Yelp Dataset

### Import Libraries

In [ ]:
# ==========Data Wrangling===========
import os
from itertools import chain
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import random

# =======For Preprocessing=======
import xlrd
import string
import re
import langid
from transformers import *
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# ==========For Model===========
from transformers import BertForSequenceClassification # Model
from transformers import AdamW, get_linear_schedule_with_warmup # Optimizer
from sklearn.metrics import f1_score  # Metrics


# ===========PYTORCH===========
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# ============NLTK============
import nltk
from nltk.collocations import *
from nltk.stem.porter import *
from nltk.probability import *
from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer
from nltk.tokenize import RegexpTokenizer
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

### EDA

In [ ]:
yelp_pd = pd.read_excel("../data_set/Yelp Labelled Review Dataset with Sentiments and Features.xlsx1", header=0)

In [3]:
yelp_pd.head()

,User_id,Product_id,Rating,Date,Review,Spam(1) and Not Spam(0),Sentiment,Features
0,923,0,3,2014-01-30,The food at snack is a selection of popular Gr...,1,Positive,"['appetizer tray', 'greek salad', 'main courses']"
1,924,0,3,2011-05-05,This little place in Soho is wonderful. I had ...,1,Positive,"['little place', 'soho', 'lamb sandwich', 'soh..."
2,925,0,4,2011-12-30,ordered lunch for 15 from Snack last Friday. Ã...,1,Positive,"['snack', 'regular company lunch list']"
3,926,0,4,2012-10-04,This is a beautiful quaint little restaurant o...,1,Positive,"['beautiful quaint', 'pretty street', 'great p..."
4,927,0,4,2014-02-06,Snack is great place for a Ã‚Â casual sit down...,1,Positive,"['snack', 'great place', 'Ã¢ casual', 'cold wi..."


In [4]:
yelp_pd.tail()

,User_id,Product_id,Rating,Date,Review,Spam(1) and Not Spam(0),Sentiment,Features
355205,161146,349,1,2012-10-04,The aircondition makes so much noise and its ...,0,Negative,[]
355206,116424,349,1,2013-05-27,Even though the pictures show very clean room...,0,Negative,"['clean rooms', 'actual room', 'o clock']"
355207,161147,349,2,2011-03-03,Backyard of the hotel is total mess shouldn t...,0,Negative,"['backyard', 'total mess shouldn t']"
355208,97930,349,2,2014-07-29,You When I booked with your company on line y...,0,Negative,"['s room', 'villa suite theough', 'wife s 40th..."
355209,5260,349,1,2013-02-07,My room was dirty and I was afraid to walk ba...,0,Negative,"['white furniture', 'angry dog', 'shower drain..."


In [5]:
yelp_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355210 entries, 0 to 355209
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   User_id                  355210 non-null  int64         
 1   Product_id               355210 non-null  int64         
 2   Rating                   355210 non-null  int64         
 3   Date                     355210 non-null  datetime64[ns]
 4   Review                   355210 non-null  object        
 5   Spam(1) and Not Spam(0)  355210 non-null  int64         
 6   Sentiment                355210 non-null  object        
 7   Features                 355210 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 21.7+ MB


In [6]:
yelp_reviews_pd = yelp_pd[["Review","Spam(1) and Not Spam(0)"]]

In [7]:
def remove_whitespace(text):
    """Remove Whitespace"""
    if isinstance(text,str):
        return " ".join(text.split())
    else:
        return " "

In [8]:
# yelp_reviews_pd.head()['Review'].apply(remove_whitespace)

In [9]:
yelp_reviews_pd['Review'] = yelp_reviews_pd['Review'].apply(remove_whitespace)
yelp_reviews_pd['Review'] = yelp_reviews_pd['Review'].str.lower()

<ipython-input-9-14ab0e4039fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yelp_reviews_pd['Review'] = yelp_reviews_pd['Review'].apply(remove_whitespace)
<ipython-input-9-14ab0e4039fe>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yelp_reviews_pd['Review'] = yelp_reviews_pd['Review'].str.lower()


In [10]:
yelp_reviews_pd['Review'].head()

0    the food at snack is a selection of popular gr...
1    this little place in soho is wonderful. i had ...
2    ordered lunch for 15 from snack last friday. ã...
3    this is a beautiful quaint little restaurant o...
4    snack is great place for a ã‚â casual sit down...
Name: Review, dtype: object

In [11]:
df =yelp_reviews_pd

In [12]:
df.columns = ['review', 'label']

In [13]:
# Decide `cuda` or `cpu`
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [14]:
device

'cuda'

In [15]:
# Split Trainging, Validation Dataset, with 15% splitting.
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['label', 'data_type']).count()

<ipython-input-15-b5d5653f7743>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_type'] = ['not_set']*df.shape[0]
C:\opt\software\Anacoda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


review
label data_type        
0     train      271215
      val         47862
1     train       30713
      val          5420

In [19]:
# 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
# Wrap training dataset
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].review.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

# Wrap validation dataset
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].review.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


### Model Training

In [21]:
# Generate BERT model.
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",  # Scientific Specific Tokenizer
                                                      num_labels=2,
                                                      output_attentions=False,
                                                      output_hidden_states=False).cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
# Batch Size
batch_size = 3

# Training DataLoader
dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

# Validation DataLoader
dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [23]:
# AdamW as Optimizer
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

# Epochs
epochs = 8# Better change to 5

# Warmup Algorithm for Optimization.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [24]:
def f1_score_func(preds, labels):
    """F1-Score Metrics"""
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    """Accuracy for Classes"""
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [25]:
# Set Random Seeds.
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):
    """
    Evaluation for Val Dataset
    Return: Loss for Val, Prediction, Grand Truth
    """
    # Close Gradient Desc
    model.eval()
    
    # Calculate the Loss for Val.
    loss_val_total = 0
    predictions, true_vals = [], []
    
    # Traverse the Val Data.
    for batch in dataloader_val:
        
        batch = tuple(b.cuda() for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        # Does not calculate the Gradient.
        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    
    # 
    return loss_val_avg, predictions, true_vals

In [ ]:
# Train the Model.
for epoch in tqdm(range(1, epochs+1)):
    
    # Open for Gradient Calculation.
    model.train()
    
    # Loss for Train
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        # Clear the Current Gradient
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        # Adding the Loss
        loss_train_total += loss.item()
        
        # Back Propagation
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Step for Optimizer
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    # Evaluation for Val Dataset.
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.45929818270800143
Validation loss: 0.47324350842297114
F1 Score (Weighted): 0.8501411485205623



Epoch 2
Training loss: 0.4621436762859857
Validation loss: 0.47602949271269174
F1 Score (Weighted): 0.8501411485205623



Epoch 3
Training loss: 0.45832859668113846
Validation loss: 0.4100581753007318
F1 Score (Weighted): 0.8501411485205623



Epoch 4
Training loss: 0.45250498087408386
Validation loss: 0.45503736005672424
F1 Score (Weighted): 0.8501411485205623



Epoch 5
Training loss: 0.44910261330340695
Validation loss: 0.408989434822283
F1 Score (Weighted): 0.8501411485205623


### Prediction

In [ ]:
# Load Test Dataset.
test_df = pd.read_csv('test_data.csv')
test_df['abstract'] = test_df['abstract'].apply(remove_whitespace)
test_df['abstract'] = test_df['abstract'].str.lower()

In [ ]:
test_df.columns = ['train_id', 'Title']
test_df['label'] = 0

In [ ]:
# Wrap up Test Data.
test_data = tokenizer.batch_encode_plus(
    test_df.Title.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

input_ids_test = test_data['input_ids']
attention_masks_test = test_data['attention_mask']
labels_test = torch.tensor(test_df.label.values)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

# Test DataLoader
dataloader_test = DataLoader(dataset_test, 
                              sampler=SequentialSampler(dataset_test), 
                              batch_size=15)

In [ ]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_2.model', map_location=torch.device('cuda')))

# Generate Prediction
_, predictions, true_vals = evaluate(dataloader_test)

In [ ]:
# Inversely Transfer to ClassNames.
label_dict_inverse = {v: k for k, v in label_dict.items()}
preds_flat = np.argmax(predictions, axis=1).flatten()
test_df['predict']= preds_flat
test_df['predict'] = test_df['predict'].replace(label_dict_inverse)

In [ ]:
test_df.drop(['Title','label'], axis=1, inplace=True)
test_df.columns = ['test_id', 'label']
test_df.to_csv('pred_labels.csv', index=False, encoding='utf-8')

In [ ]:
test_df

## References

[1] BERT Text Classification Using Pytorch. https://towardsdatascience.com/bert-text-classification-using-pytorch-723dfb8b6b5b